In [1]:
import requests
import json
import pandas as pd

# Import dotenv package for setting environment variables 
from dotenv import load_dotenv

# Import os package
import os

In [3]:
df = pd.read_parquet('lyft_Dec_23.parquet')
bins_df=pd.read_csv('taxi_zone_lookup.csv')
df.sort_index(inplace=True)
display(df.head())
df.tail()

,pickup_datetime,PULocationID,DOLocationID,trip_miles,trip_time,driver_pay,Total_Passenger_Cost
17,2023-12-01 00:50:08,256,132,17.227,2271,44.32,65.87
18,2023-12-01 00:09:21,40,97,1.934,677,8.98,15.73
19,2023-12-01 00:32:32,148,79,1.313,655,7.96,15.38
20,2023-12-01 00:47:13,107,256,4.059,1184,16.60,25.78
27,2023-12-01 00:46:08,129,138,1.436,189,5.47,11.99


,pickup_datetime,PULocationID,DOLocationID,trip_miles,trip_time,driver_pay,Total_Passenger_Cost
20516282,2023-12-31 23:03:55,164,234,1.246,1166,12.72,27.35
20516283,2023-12-31 23:31:29,234,164,0.979,777,8.93,30.28
20516290,2023-12-31 23:09:22,17,62,1.878,529,7.50,11.89
20516291,2023-12-31 23:23:47,62,17,1.977,726,9.51,28.94
20516292,2023-12-31 23:39:27,17,49,1.581,518,7.00,11.81


In [13]:
bins_df = bins_df.drop(bins_df[bins_df['product_name'] == 'Coca Cola'].index)

<class 'pandas.core.frame.DataFrame'>
Index: 6242671 entries, 17 to 20516292
Data columns (total 7 columns):
 #   Column                Dtype         
---  ------                -----         
 0   pickup_datetime       datetime64[us]
 1   PULocationID          int32         
 2   DOLocationID          int32         
 3   trip_miles            float64       
 4   trip_time             int64         
 5   driver_pay            float64       
 6   Total_Passenger_Cost  float64       
dtypes: datetime64[us](1), float64(3), int32(2), int64(1)
memory usage: 333.4 MB


In [25]:
out_unknown_df=df.loc[df['PULocationID'].isin([1])]
out_unknown_df.head()

,pickup_datetime,PULocationID,DOLocationID,trip_miles,trip_time,driver_pay,Total_Passenger_Cost


In [26]:
in_unknown_df=df.loc[df['DOLocationID'].isin([1])]
in_unknown_df

,pickup_datetime,PULocationID,DOLocationID,trip_miles,trip_time,driver_pay,Total_Passenger_Cost
20022,2023-12-01 00:53:46,230,1,22.167,3699,72.15,107.02
38307,2023-12-01 01:57:13,164,1,17.235,1744,43.96,78.52
41337,2023-12-01 02:53:53,13,1,20.701,2040,51.28,84.50
42599,2023-12-01 02:55:00,176,1,14.482,1473,36.91,83.95
44277,2023-12-01 02:59:00,256,1,25.223,2114,55.16,84.69
...,...,...,...,...,...,...,...
20349644,2023-12-31 20:54:50,231,1,12.792,1185,31.79,72.99
20358543,2023-12-31 20:22:12,161,1,15.461,2496,47.66,100.27
20407018,2023-12-31 21:36:08,48,1,15.184,1571,39.59,78.71
20410457,2023-12-31 21:26:24,68,1,15.883,1518,62.18,82.85


In [6]:
bins_df

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NaN,NaN


In [35]:
bins_df.drop(columns="service_zone")

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights
...,...,...,...
260,261,Manhattan,World Trade Center
261,262,Manhattan,Yorkville East
262,263,Manhattan,Yorkville West
263,264,Unknown,NaN


In [38]:
bins_df['Borough'] = bins_df['Borough'].fillna("Outside of NYC")

In [46]:
bins_df = bins_df[bins_df.Borough != "Unknown"]

In [47]:
bins_df["Borough"].unique()

array(['EWR', 'Queens', 'Bronx', 'Manhattan', 'Staten Island', 'Brooklyn',
       'Outside of NYC'], dtype=object)

In [67]:
group=bins_df.groupby(by=["Borough"])
group.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone


In [51]:
#Establishing the bins of each location ID
queens_bin=bins_df.loc[bins_df['Borough'] == 'Queens']

bronx_bin=bins_df.loc[bins_df['Borough'] == 'Bronx']

mh_bin=bins_df.loc[bins_df['Borough'] == 'Manhattan']

si_bin=bins_df.loc[bins_df['Borough'] == 'Staten Island']

brooklyn_bin=bins_df.loc[bins_df['Borough'] == 'Brooklyn']

out_of_nyc_bin=bins_df.loc[bins_df['Borough']== 'Outside of NYC']

newark_bin=bins_df.loc[bins_df['Borough'] == 'EWR']

bins_list=[queens_bin ,bronx_bin,mh_bin,si_bin,brooklyn_bin,out_of_nyc_bin,
           newark_bin]
queens_bin


,LocationID,Borough,Zone,service_zone
1,2,Queens,Jamaica Bay,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone
...,...,...,...,...
225,226,Queens,Sunnyside,Boro Zone
251,252,Queens,Whitestone,Boro Zone
252,253,Queens,Willets Point,Boro Zone
257,258,Queens,Woodhaven,Boro Zone


In [59]:
#Dropping Zone and service_zone columns
for bin in bins_list:
    bin.drop(columns=['Zone', 'service_zone'],inplace=True)

bins_list

C:\Users\Andrew\AppData\Local\Temp\ipykernel_5940\2210486645.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bin.drop(columns=['Zone', 'service_zone'],inplace=True)
C:\Users\Andrew\AppData\Local\Temp\ipykernel_5940\2210486645.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bin.drop(columns=['Zone', 'service_zone'],inplace=True)
C:\Users\Andrew\AppData\Local\Temp\ipykernel_5940\2210486645.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

[     LocationID Borough
 1             2  Queens
 6             7  Queens
 7             8  Queens
 8             9  Queens
 9            10  Queens
 ..          ...     ...
 225         226  Queens
 251         252  Queens
 252         253  Queens
 257         258  Queens
 259         260  Queens
 
 [69 rows x 2 columns],
      LocationID Borough
 2             3   Bronx
 17           18   Bronx
 19           20   Bronx
 30           31   Bronx
 31           32   Bronx
 45           46   Bronx
 46           47   Bronx
 50           51   Bronx
 57           58   Bronx
 58           59   Bronx
 59           60   Bronx
 68           69   Bronx
 77           78   Bronx
 80           81   Bronx
 93           94   Bronx
 118         119   Bronx
 125         126   Bronx
 135         136   Bronx
 146         147   Bronx
 158         159   Bronx
 166         167   Bronx
 167         168   Bronx
 168         169   Bronx
 173         174   Bronx
 181         182   Bronx
 182         183   Bronx

In [70]:
brooklyn_bin

,LocationID,Borough
10,11,Brooklyn
13,14,Brooklyn
16,17,Brooklyn
20,21,Brooklyn
21,22,Brooklyn
...,...,...
226,227,Brooklyn
227,228,Brooklyn
254,255,Brooklyn
255,256,Brooklyn
